# Public transport data from Geofabrik on IDF scale
This data is later subsetted to GPM in QGIS.
WARNING: I DID NOT RERUN THIS SINCE CHANGING EPSGs, MAKE SURE TO RUN IT AGAIN!

In [3]:
import shapely.wkb
import pandas as pd 
import geopandas as gpd
from pyrosm import OSM
from pyrosm import get_data
import matplotlib.pyplot as plt
import networkx as nx

plt.rcParams['figure.figsize'] = (15, 15)
crs_fr = 2154


In [4]:
# Get data on IDF level
fp = get_data("Ile_de_France")

# Initialize the OSM parser object
osm = OSM(fp)

Downloaded Protobuf data 'ile-de-france-latest.osm.pbf' (281.26 MB) to:
'/var/folders/2m/lkz691q11b72bpprh6_v34_h0000gn/T/pyrosm/ile-de-france-latest.osm.pbf'


In [5]:
import osmium
import shapely.wkb
import pandas as pd
import geopandas as gpd


def merge_two_dicts(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

class AdminAreaHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)

        self.areas = []
        self.wkbfab = osmium.geom.WKBFactory()

    def area(self, a):            
        if  "admin_level" in a.tags:

            wkbshape = self.wkbfab.create_multipolygon(a)
            shapely_obj = shapely.wkb.loads(wkbshape, hex=True)

            area = { "id": a.id, "geo": shapely_obj }
            area = merge_two_dicts(area, a.tags)

            self.areas.append(area)
        
handler = AdminAreaHandler()

# path to file to local drive
# download from https://download.geofabrik.de/index.html
#osm_file = osm

# start data file processing
handler.apply_file(fp, locations=True, idx='flex_mem')

df = pd.DataFrame(handler.areas)

In [6]:
gdf = gpd.GeoDataFrame(df, geometry="geo")
gdf.head(1)

,id,geo,admin_level,boundary,name,population,population:date,postal_code,ref:INSEE,source:population,...,name:gd,manufacturer,source:name:br,name:az,name:pt,name:sr-Latn,name:zh-Hant,ref:nuts:1,ref:nuts:2,source:name:oc
0,55263,"MULTIPOLYGON (((2.12315 48.75362, 2.12323 48.7...",8,administrative,Les Loges-en-Josas,1629,2019,78350,78343,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Bus

In [ ]:
# Get bus stops and routes
# =====================================

custom_filter = {'public_transport':['stop_position'],'route':['bus']}
bus_IDF = osm.get_pois(custom_filter=custom_filter)
bus_IDF = bus_stops_IDF.to_crs(crs_fr)

bus_IDF["bus_stop"] = bus_stops_IDF["public_transport"]
bus_IDF["bus_route"] = bus_stops_IDF["route"]

IOStream.flush timed out


In [ ]:
# Plot for test 
ax = plt.axes()

bus_IDF.plot(column='bus_route', ax=ax, linewidth = 0.5, edgecolor = 'red')
bus_IDF.plot(column='bus_stop', ax=ax, markersize=1.5)

In [ ]:
bus_IDF.head()

In [ ]:
# Save to geojson to use in QGIS 
#bus_IDF = pois
bus_IDF.to_file('data/processed/bus_IDF.geojson', driver='GeoJSON')

# Metro

In [7]:
# Get metro stops and routes
# =====================================

# TODO find out correct keywords
custom_filter = {'public_transport':['stop_position'],'route':['metro']}
metro_stops_IDF = osm.get_pois(custom_filter=custom_filter)

AttributeError: module 'shapely.geos' has no attribute 'lgeos'